# Parte 1: PREPARACIÓN DE DATOS RETAIL

In [20]:
import pandas
from datetime import datetime
import pymysql.cursors
import pymysql
from sqlalchemy import create_engine

In [21]:
#Cargar información
orders=pandas.read_excel('input/RetailData_Colfondos_Test.xls',sheet_name='Orders',header=0)
customers=pandas.read_excel('input/RetailData_Colfondos_Test.xls',sheet_name='Customers',header=0)
returns=pandas.read_excel('input/RetailData_Colfondos_Test.xls',sheet_name='Returns',header=0)
region_managers=pandas.read_excel('input/RetailData_Colfondos_Test.xls',sheet_name='Region Managers',header=0)

##Tratamiento Orders

In [22]:
orders.shape[0]

10042

In [23]:
orders=orders.dropna(subset=['Order Date','Ship Date','Customer ID'])
orders.shape[0]

10023

In [24]:
columnas=['Product ID','Category','Sub-Category','Product Name']
orders[columnas] = orders[columnas].fillna('Otros')

In [25]:
#orders[orders['Product ID'].isna()]
orders[orders['Product ID']=='Otros']

,Row ID,Order ID,Order Date,Ship Date,Ship Mode,Customer ID,Product ID,Category,Sub-Category,Product Name,Sales,Quantity,Discount
9964,9965,CA-2013-146374,2013-12-06 00:00:00,2013-12-11 00:00:00,Second Class,HE-14800,Otros,Otros,Otros,Otros,13.40,1,0.0
9965,9966,CA-2013-146374,2013-12-06 00:00:00,2013-12-11 00:00:00,Second Class,HE-14800,Otros,Otros,Otros,Otros,4.98,1,0.0
9966,9967,CA-2013-146374,2013-12-06 00:00:00,2013-12-11 00:00:00,Second Class,HE-14800,Otros,Otros,Otros,Otros,109.69,7,0.0
9967,9968,CA-2014-153871,2014-12-12 00:00:00,2014-12-18 00:00:00,Standard Class,RB-19435,Otros,Otros,Otros,Otros,40.20,5,0.0
9974,9975,US-2013-103674,2013-12-07 00:00:00,2013-12-11 00:00:00,Standard Class,AP-10720,Otros,Otros,Otros,Otros,18.69,7,0.0
9975,9976,US-2013-103674,2013-12-07 00:00:00,2013-12-11 00:00:00,Standard Class,AP-10720,Otros,Otros,Otros,Otros,13.36,2,0.0


In [26]:
orders[orders['Quantity']<0]

,Row ID,Order ID,Order Date,Ship Date,Ship Mode,Customer ID,Product ID,Category,Sub-Category,Product Name,Sales,Quantity,Discount
10015,10016,CA-2014-477461,2014-12-12 00:00:00,2014-12-18 00:00:00,Standard Class,RB-19435,OFF-BI-10004209,Office Supplies,Binders,"Fellowes Twister Kit, Gray/Clear, 3/pkg",337.0,-5,0.0
10016,10017,CA-2014-477462,2014-12-12 00:00:00,2014-12-18 00:00:00,Standard Class,RB-19435,OFF-BI-10004600,Office Supplies,Binders,Ibico Ibimaster 300 Manual Binding System,386.0,-2,0.0
10017,10018,CA-2014-477463,2014-12-12 00:00:00,2014-12-18 00:00:00,Standard Class,RB-19435,OFF-AP-10003622,Office Supplies,Appliances,"Bravo II Megaboss 12-Amp Hard Body Upright, Re...",317.0,-7,0.0
10018,10019,CA-2012-335601,2012-06-28 00:00:00,2012-07-02 00:00:00,Standard Class,MP-17470,OFF-BI-10002867,Office Supplies,Binders,GBC Recycled Regency Composition Covers,388.0,-2,0.0


In [27]:
columnas=['Sales','Quantity']
mask = orders[columnas].lt(0).any(axis=1)
orders = orders[~mask]

In [28]:
columnas=['Category','Sub-Category']
orders[columnas]=orders[columnas].apply(lambda x: x.str.upper())

In [29]:
orders

,Row ID,Order ID,Order Date,Ship Date,Ship Mode,Customer ID,Product ID,Category,Sub-Category,Product Name,Sales,Quantity,Discount
0,1,CA-2013-152156,2013-11-09 00:00:00,2013-11-12 00:00:00,Second Class,CG-12520,FUR-BO-10001798,FURNITURE,BOOKCASES,Bush Somerset Collection Bookcase,261.9600,2,0.00
1,2,CA-2013-152156,2013-11-09 00:00:00,2013-11-12 00:00:00,Second Class,CG-12520,FUR-CH-10000454,FURNITURE,CHAIRS,"Hon Deluxe Fabric Upholstered Stacking Chairs,...",731.9400,3,0.00
2,3,CA-2013-138688,2013-06-13 00:00:00,2013-06-17 00:00:00,Second Class,DV-13045,OFF-LA-10000240,OFFICE SUPPLIES,LABELS,Self-Adhesive Address Labels for Typewriters b...,14.6200,2,0.00
3,4,US-2012-108966,2012-10-11 00:00:00,2012-10-18 00:00:00,Standard Class,SO-20335,FUR-TA-10000577,FURNITURE,TABLES,Bretford CR4500 Series Slim Rectangular Table,957.5775,5,0.45
4,5,US-2012-108966,2012-10-11 00:00:00,2012-10-18 00:00:00,Standard Class,SO-20335,OFF-ST-10000760,OFFICE SUPPLIES,STORAGE,Eldon Fold 'N Roll Cart System,22.3680,2,0.20
...,...,...,...,...,...,...,...,...,...,...,...,...,...
10037,10038,CA-2011-352616,2011-01-22 00:00:00,2011-01-24 00:00:00,Second Class,TB-21400,FUR-FU-10001889,FURNITURE,FURNISHINGS,Ultra Door Pull Handle,344.0000,3,0.20
10038,10039,CA-2014-377743,2014-02-27 00:00:00,2014-03-04 00:00:00,Standard Class,DB-13060,FUR-FU-10000747,FURNITURE,FURNISHINGS,Tenex B1-RE Series Chair Mats for Low Pile Car...,50.0000,2,0.00
10039,10040,CA-2014-377744,2014-02-27 00:00:00,2014-03-04 00:00:00,Standard Class,DB-13060,TEC-PH-10003645,TECHNOLOGY,PHONES,Aastra 57i VoIP phone,397.0000,2,0.20
10040,10041,CA-2014-377745,2014-02-27 00:00:00,2014-03-04 00:00:00,Standard Class,DB-13060,OFF-PA-10004041,OFFICE SUPPLIES,PAPER,"It's Hot Message Books with Stickers, 2 3/4"" x 5""",106.0000,4,0.00


In [30]:
def safe_to_datetime(date_str):
    try:
        return pandas.to_datetime(date_str, errors='coerce')
    except Exception as e:
        print(f"Error al convertir la fecha {date_str}: {e}")
        return pandas.NaT  # Retorna un valor nulo en caso de error
    
orders['Order Date'] = orders['Order Date'].apply(safe_to_datetime)

# Filtrar filas con fechas válidas
orders = orders.dropna(subset=['Order Date'])

In [31]:
orders.shape[0]

10001

In [32]:
orders['Año'] = orders['Order Date'].dt.year
orders['Mes'] = orders['Order Date'].dt.month
orders['Dia']=1
orders

C:\Users\jerf_\AppData\Local\Temp/ipykernel_72576/825309717.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  orders['Año'] = orders['Order Date'].dt.year
C:\Users\jerf_\AppData\Local\Temp/ipykernel_72576/825309717.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  orders['Mes'] = orders['Order Date'].dt.month
C:\Users\jerf_\AppData\Local\Temp/ipykernel_72576/825309717.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer]

,Row ID,Order ID,Order Date,Ship Date,Ship Mode,Customer ID,Product ID,Category,Sub-Category,Product Name,Sales,Quantity,Discount,Año,Mes,Dia
0,1,CA-2013-152156,2013-11-09,2013-11-12 00:00:00,Second Class,CG-12520,FUR-BO-10001798,FURNITURE,BOOKCASES,Bush Somerset Collection Bookcase,261.9600,2,0.00,2013,11,1
1,2,CA-2013-152156,2013-11-09,2013-11-12 00:00:00,Second Class,CG-12520,FUR-CH-10000454,FURNITURE,CHAIRS,"Hon Deluxe Fabric Upholstered Stacking Chairs,...",731.9400,3,0.00,2013,11,1
2,3,CA-2013-138688,2013-06-13,2013-06-17 00:00:00,Second Class,DV-13045,OFF-LA-10000240,OFFICE SUPPLIES,LABELS,Self-Adhesive Address Labels for Typewriters b...,14.6200,2,0.00,2013,6,1
3,4,US-2012-108966,2012-10-11,2012-10-18 00:00:00,Standard Class,SO-20335,FUR-TA-10000577,FURNITURE,TABLES,Bretford CR4500 Series Slim Rectangular Table,957.5775,5,0.45,2012,10,1
4,5,US-2012-108966,2012-10-11,2012-10-18 00:00:00,Standard Class,SO-20335,OFF-ST-10000760,OFFICE SUPPLIES,STORAGE,Eldon Fold 'N Roll Cart System,22.3680,2,0.20,2012,10,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10037,10038,CA-2011-352616,2011-01-22,2011-01-24 00:00:00,Second Class,TB-21400,FUR-FU-10001889,FURNITURE,FURNISHINGS,Ultra Door Pull Handle,344.0000,3,0.20,2011,1,1
10038,10039,CA-2014-377743,2014-02-27,2014-03-04 00:00:00,Standard Class,DB-13060,FUR-FU-10000747,FURNITURE,FURNISHINGS,Tenex B1-RE Series Chair Mats for Low Pile Car...,50.0000,2,0.00,2014,2,1
10039,10040,CA-2014-377744,2014-02-27,2014-03-04 00:00:00,Standard Class,DB-13060,TEC-PH-10003645,TECHNOLOGY,PHONES,Aastra 57i VoIP phone,397.0000,2,0.20,2014,2,1
10040,10041,CA-2014-377745,2014-02-27,2014-03-04 00:00:00,Standard Class,DB-13060,OFF-PA-10004041,OFFICE SUPPLIES,PAPER,"It's Hot Message Books with Stickers, 2 3/4"" x 5""",106.0000,4,0.00,2014,2,1


In [33]:
#orders['Fecha'] = pandas.to_datetime(orders[['Año', 'Mes', 'Dia']],format='%Y-%m-%d', errors='coerce')
def crear_fecha(row):
    try:
        return datetime(year=row['Año'], month=row['Mes'], day=row['Dia'])
    except ValueError:
        return pd.NaT  # Retorna un valor nulo en caso de error

# Aplicar la función para crear la columna de fecha
orders['Order Date'] = orders.apply(crear_fecha, axis=1)
orders.drop(columns=['Año', 'Mes','Dia'], inplace=True)

C:\Users\jerf_\AppData\Local\Temp/ipykernel_72576/2262886720.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  orders['Order Date'] = orders.apply(crear_fecha, axis=1)
C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\frame.py:4906: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


In [34]:
orders

,Row ID,Order ID,Order Date,Ship Date,Ship Mode,Customer ID,Product ID,Category,Sub-Category,Product Name,Sales,Quantity,Discount
0,1,CA-2013-152156,2013-11-01,2013-11-12 00:00:00,Second Class,CG-12520,FUR-BO-10001798,FURNITURE,BOOKCASES,Bush Somerset Collection Bookcase,261.9600,2,0.00
1,2,CA-2013-152156,2013-11-01,2013-11-12 00:00:00,Second Class,CG-12520,FUR-CH-10000454,FURNITURE,CHAIRS,"Hon Deluxe Fabric Upholstered Stacking Chairs,...",731.9400,3,0.00
2,3,CA-2013-138688,2013-06-01,2013-06-17 00:00:00,Second Class,DV-13045,OFF-LA-10000240,OFFICE SUPPLIES,LABELS,Self-Adhesive Address Labels for Typewriters b...,14.6200,2,0.00
3,4,US-2012-108966,2012-10-01,2012-10-18 00:00:00,Standard Class,SO-20335,FUR-TA-10000577,FURNITURE,TABLES,Bretford CR4500 Series Slim Rectangular Table,957.5775,5,0.45
4,5,US-2012-108966,2012-10-01,2012-10-18 00:00:00,Standard Class,SO-20335,OFF-ST-10000760,OFFICE SUPPLIES,STORAGE,Eldon Fold 'N Roll Cart System,22.3680,2,0.20
...,...,...,...,...,...,...,...,...,...,...,...,...,...
10037,10038,CA-2011-352616,2011-01-01,2011-01-24 00:00:00,Second Class,TB-21400,FUR-FU-10001889,FURNITURE,FURNISHINGS,Ultra Door Pull Handle,344.0000,3,0.20
10038,10039,CA-2014-377743,2014-02-01,2014-03-04 00:00:00,Standard Class,DB-13060,FUR-FU-10000747,FURNITURE,FURNISHINGS,Tenex B1-RE Series Chair Mats for Low Pile Car...,50.0000,2,0.00
10039,10040,CA-2014-377744,2014-02-01,2014-03-04 00:00:00,Standard Class,DB-13060,TEC-PH-10003645,TECHNOLOGY,PHONES,Aastra 57i VoIP phone,397.0000,2,0.20
10040,10041,CA-2014-377745,2014-02-01,2014-03-04 00:00:00,Standard Class,DB-13060,OFF-PA-10004041,OFFICE SUPPLIES,PAPER,"It's Hot Message Books with Stickers, 2 3/4"" x 5""",106.0000,4,0.00


##Tratamiento Customers

In [35]:
customers = customers.drop_duplicates()

In [36]:
customers

,Customer ID,Customer Name,Segment,Country,City,State,Postal Code,Region
0,CG-12520,Claire Gute,Consumer,United States,Henderson,Kentucky,42420,South
2,DV-13045,Darrin Van Huff,Corporate,United States,Los Angeles,California,90036,West
3,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,Florida,33311,South
5,BH-11710,Brosina Hoffman,Consumer,United States,Los Angeles,California,90032,West
12,AA-10480,Andrew Allen,Consumer,United States,Concord,North Carolina,28027,South
...,...,...,...,...,...,...,...,...
9986,ML-17410,Maris LaWare,Consumer,United States,Los Angeles,California,90008,West
9987,RA-19885,Ruben Ausman,Corporate,United States,Athens,Georgia,30605,South
9989,TB-21400,Tom Boeckenhauer,Consumer,United States,Miami,Florida,33180,South
9990,DB-13060,Dave Brooks,Consumer,United States,Costa Mesa,California,92627,West


In [37]:
# Paso 1: Contar las apariciones de cada ciudad para cada id
conteo_ciudades = customers.groupby(['Customer ID', 'City']).size().reset_index(name='conteo')

# Paso 2: Encontrar la ciudad con el mayor conteo para cada id
ciudad_max_conteo = conteo_ciudades.loc[conteo_ciudades.groupby('Customer ID')['conteo'].idxmax()]

# Paso 3: Unir el resultado con el DataFrame original para obtener el registro completo
customers = customers.merge(ciudad_max_conteo[['Customer ID', 'City']], on=['Customer ID', 'City'])
customers=customers.drop_duplicates(subset='Customer ID')

In [38]:
customers.sort_values(by=['Customer ID'])

,Customer ID,Customer Name,Segment,Country,City,State,Postal Code,Region
188,AA-10315,Alex Avila,Consumer,United States,San Francisco,California,94109,West
331,AA-10375,Allen Armold,Consumer,United States,Atlanta,Georgia,30318,South
0,AA-10480,Andrew Allen,Consumer,United States,Concord,North Carolina,28027,South
515,AA-10645,Anna Andreadi,Consumer,United States,Chester,Pennsylvania,19013,East
870,AB-10015,Aaron Bergman,Consumer,United States,Arlington,Texas,76017,Central
...,...,...,...,...,...,...,...,...
566,XP-21865,Xylona Preis,Consumer,United States,Los Angeles,California,90049,West
649,YC-21895,Yoseph Carroll,Corporate,United States,Dallas,Texas,75217,Central
206,YS-21880,Yana Sorensen,Corporate,United States,Burlington,North Carolina,27217,South
612,ZC-21910,Zuschuss Carroll,Consumer,United States,Aurora,Colorado,80013,West


Consolidar información cruzando todas las tablas.

In [39]:
consolidada=pandas.merge(orders, customers, how='left', on=['Customer ID'])

In [40]:
consolidada.shape[0]

10001

In [41]:
consolidada=pandas.merge(consolidada, region_managers, how='left', on=['Region'])
consolidada.shape[0]

10001

In [42]:
consolidada=pandas.merge(consolidada, returns, how='left', on=['Order ID'])
consolidada.shape[0]

10001

In [43]:
consolidada['Intresos_Totales']=consolidada['Sales']*consolidada['Quantity']
consolidada

,Row ID,Order ID,Order Date,Ship Date,Ship Mode,Customer ID,Product ID,Category,Sub-Category,Product Name,...,Customer Name,Segment,Country,City,State,Postal Code,Region,Regional Manager,Returned,Intresos_Totales
0,1,CA-2013-152156,2013-11-01,2013-11-12 00:00:00,Second Class,CG-12520,FUR-BO-10001798,FURNITURE,BOOKCASES,Bush Somerset Collection Bookcase,...,Claire Gute,Consumer,United States,Dallas,Texas,75217,Central,Kelly Williams,NaN,523.9200
1,2,CA-2013-152156,2013-11-01,2013-11-12 00:00:00,Second Class,CG-12520,FUR-CH-10000454,FURNITURE,CHAIRS,"Hon Deluxe Fabric Upholstered Stacking Chairs,...",...,Claire Gute,Consumer,United States,Dallas,Texas,75217,Central,Kelly Williams,NaN,2195.8200
2,3,CA-2013-138688,2013-06-01,2013-06-17 00:00:00,Second Class,DV-13045,OFF-LA-10000240,OFFICE SUPPLIES,LABELS,Self-Adhesive Address Labels for Typewriters b...,...,Darrin Van Huff,Corporate,United States,Jacksonville,Florida,32216,South,Cassandra Brandow,NaN,29.2400
3,4,US-2012-108966,2012-10-01,2012-10-18 00:00:00,Standard Class,SO-20335,FUR-TA-10000577,FURNITURE,TABLES,Bretford CR4500 Series Slim Rectangular Table,...,Sean O'Donnell,Consumer,United States,Columbia,Tennessee,38401,South,Cassandra Brandow,NaN,4787.8875
4,5,US-2012-108966,2012-10-01,2012-10-18 00:00:00,Standard Class,SO-20335,OFF-ST-10000760,OFFICE SUPPLIES,STORAGE,Eldon Fold 'N Roll Cart System,...,Sean O'Donnell,Consumer,United States,Columbia,Tennessee,38401,South,Cassandra Brandow,NaN,44.7360
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9996,10038,CA-2011-352616,2011-01-01,2011-01-24 00:00:00,Second Class,TB-21400,FUR-FU-10001889,FURNITURE,FURNISHINGS,Ultra Door Pull Handle,...,Tom Boeckenhauer,Consumer,United States,Seattle,Washington,98105,West,Anna Andreadi,NaN,1032.0000
9997,10039,CA-2014-377743,2014-02-01,2014-03-04 00:00:00,Standard Class,DB-13060,FUR-FU-10000747,FURNITURE,FURNISHINGS,Tenex B1-RE Series Chair Mats for Low Pile Car...,...,Dave Brooks,Consumer,United States,Chicago,Illinois,60653,Central,Kelly Williams,NaN,100.0000
9998,10040,CA-2014-377744,2014-02-01,2014-03-04 00:00:00,Standard Class,DB-13060,TEC-PH-10003645,TECHNOLOGY,PHONES,Aastra 57i VoIP phone,...,Dave Brooks,Consumer,United States,Chicago,Illinois,60653,Central,Kelly Williams,NaN,794.0000
9999,10041,CA-2014-377745,2014-02-01,2014-03-04 00:00:00,Standard Class,DB-13060,OFF-PA-10004041,OFFICE SUPPLIES,PAPER,"It's Hot Message Books with Stickers, 2 3/4"" x 5""",...,Dave Brooks,Consumer,United States,Chicago,Illinois,60653,Central,Kelly Williams,NaN,424.0000


In [34]:
#MySQL DB credentials
host='database-1.cwqfoh27htqv.us-east-2.rds.amazonaws.com'
port=3306
user='admin'
password='admin123'
database='dbcolfondos'

#connect to  BD
def conexionBD(user,password,host,database):
    cnx = pymysql.connections.Connection(user=user, password=password,host=host,database=database)
    cursor = cnx.cursor()
    return cursor

#import data from csv and create datawarehouse in MySQL DB
#create DB engine
engine = create_engine('mysql+pymysql://{}:{}@{}:{}/{}'.format(user,password,host,port,database))


In [41]:
#Subir informacion a BD de MYSQL
consolidada.to_sql('consolidada', engine,index=False,if_exists='replace')

In [44]:
consolidada.to_excel('consolidada_parte1.xlsx', index=False, sheet_name='Datos')